# WorldView enhancement playground

This notebook demonstrates four workflows using the project's scripts:

- Pansharpening only
- Coregistration only
- Complete pipeline (pansharpen then coregister)
- Pipeline with coregistration first

All steps use small synthetic data so the examples run quickly on a local machine. Replace the synthetic files with real data paths to run on real imagery.

In [ ]:
# Setup: imports and path adjustments
import os
import sys
from pathlib import Path
import numpy as np
import rasterio

# Ensure repo root is importable so we can import `scripts` as a package
repo_root = Path('..').resolve()
sys.path.insert(0, str(repo_root))

# Imports from project scripts
from scripts.gsa_pansharpening import process_worldview_rgb, GSA_sharpening
from scripts.coregistration import coregister_images, create_coregistration_params, prepare_stack_for_coregistration
from scripts.worldview_enhance import enhance_worldview_image

# Create example output directory
example_dir = Path('example_data')
example_dir.mkdir(exist_ok=True)

print('Example data directory:', example_dir.resolve())

In [ ]:
# Synthetic data creation
from rasterio.transform import from_origin

# deterministic seed for reproducibility
np.random.seed(42)

ms_path = str(example_dir / 'sample_ms.tif')
pan_path = str(example_dir / 'sample_pan.tif')
ref_path = str(example_dir / 'sample_reference.tif')

# Multispectral (3 bands) low-res 80x80
ms_shape = (3, 80, 80)
ms_data = (np.random.rand(*ms_shape) * 10000).astype('float32')
ms_meta = {
    'driver': 'GTiff',
    'height': ms_shape[1],
    'width': ms_shape[2],
    'count': ms_shape[0],
    'dtype': 'float32',
    'crs': 'EPSG:32633',
    'transform': from_origin(0, 80, 1.0, 1.0)
}
with rasterio.open(ms_path, 'w', **ms_meta) as dst:
    dst.write(ms_data)

# Panchromatic high-res 320x320 (4x resolution)
pan_shape = (320, 320)
pan_data = (np.random.rand(*pan_shape) * 10000).astype('float32')
pan_meta = {
    'driver': 'GTiff',
    'height': pan_shape[0],
    'width': pan_shape[1],
    'count': 1,
    'dtype': 'float32',
    'crs': 'EPSG:32633',
    'transform': from_origin(0, 80, 0.25, 0.25)
}
with rasterio.open(pan_path, 'w', **pan_meta) as dst:
    dst.write(pan_data, 1)

# Reference image (simulate Google satellite) - same resolution as pan
ref_data = (np.random.rand(*pan_shape) * 10000).astype('float32')
ref_meta = pan_meta.copy()
with rasterio.open(ref_path, 'w', **ref_meta) as dst:
    dst.write(ref_data, 1)

print('Created synthetic files:')
print('-', ms_path)
print('-', pan_path)
print('-', ref_path)

## Pansharpening only

This cell runs the GSA pansharpening implementation on the synthetic multispectral and panchromatic images and writes an output file. We print basic statistics about the result to confirm execution.

In [ ]:
# Run pansharpening only
out_pansharp = str(example_dir / 'pansharpened_only.tif')
process_worldview_rgb(ms_path, pan_path, out_pansharp)

# Quick check
with rasterio.open(out_pansharp) as src:
    arr = src.read()
    print('Pansharpen output:', out_pansharp)
    print('  shape (bands, h, w):', arr.shape)
    for i in range(arr.shape[0]):
        print(f'  band {i+1} min/max: {arr[i].min():.2f}/{arr[i].max():.2f}')


## Coregistration only

This cell demonstrates creating a stacked RGB+PAN image and coregistering it to the synthetic reference image using AROSICS. We use conservative parameters to keep runtime short.

In [ ]:
# Coregistration only: prepare stack and run AROSICS
stack_path = str(example_dir / 'rgb_pan_stack.tif')
coreg_out = str(example_dir / 'coreg_only.tif')

# Prepare stack (resample RGB to PAN resolution and stack)
print('Preparing stack ->', stack_path)
prepare_stack_for_coregistration(ms_path, pan_path, stack_path)

# Create params tuned for quick demo
params = create_coregistration_params(grid_res=5, window_size=(50,50), max_shift=10)

print('Running coregistration...')
coregister_images(ref_path, stack_path, coreg_out, params=params)
print('Coregistration output:', coreg_out)


## Complete pipeline: pansharpen then coregister

This cell runs the full pipeline: first pansharpen the multispectral with the panchromatic image, then coregister the pansharpened result to the reference image.

In [ ]:
# Run full pipeline (pansharpen then coregister)
pipeline_prefix = str(example_dir / 'pipeline')
enhance_worldview_image(ms_path, pan_path, ref_path, pipeline_prefix, coregister_first=False)

print('Pipeline outputs (prefix):', pipeline_prefix)
# show produced files (if created)
for suff in ['_sharp.tif', '_enhanced.tif', '_coreg_stack.tif', '_stack.tif']:
    p = Path(pipeline_prefix + suff)
    if p.exists():
        print(' -', str(p))


## Alternative pipeline: coregister then pansharpen

Run the reverse order (coregister first then pansharpen). This may be useful when the source MS and PAN images have different alignments that need correction prior to fusion.

In [ ]:
# Run alternative pipeline (coregister first)
altn_prefix = str(example_dir / 'pipeline_coreg_first')
enhance_worldview_image(ms_path, pan_path, ref_path, altn_prefix, coregister_first=True)

print('Alternative pipeline outputs (prefix):', altn_prefix)
for suff in ['_stack.tif', '_coreg_stack.tif', '_enhanced.tif', '_coreg.tif', '_sharp.tif']:
    p = Path(altn_prefix + suff)
    if p.exists():
        print(' -', str(p))


In [ ]:
# Final: list files produced in example_data
print('\nFiles in example_data:')
for p in sorted(example_dir.iterdir()):
    try:
        size = p.stat().st_size
    except Exception:
        size = -1
    print(f" - {p.name:40} {size:10d} bytes")
